In [1]:
import pickle
import tensorflow as tf
import sklearn
from sklearn.model_selection import train_test_split
import numpy as np
import csv
import cv2

In [2]:
samples = []
with open('./data/data/driving_log1.csv') as csvfile:
    reader = csv.reader(csvfile)
    for sample in reader:
        samples.append(sample)
        
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

In [3]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './data/data/IMG/'+batch_sample[0].split('/')[-1]
                image = cv2.imread(name)
                image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
                images.append(image)
                angle = float(batch_sample[3])
                angles.append(angle)
                
            augmented_images = []
            augmented_measurements = []
            for image, angle in zip(images, angles):
                augmented_images.append(image)
                augmented_measurements.append(angle)
                augmented_images.append(cv2.flip(image,1))
                augmented_measurements.append(angle*(-1.0))

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

In [4]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import Cropping2D

model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((70,25), (0,0)), input_shape=(3,160,320)))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))


model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch=len(train_samples), \
                    validation_data=validation_generator,nb_val_samples=len(validation_samples),nb_epoch=3)
model.save('model.h5')

Using TensorFlow backend.


Epoch 1/3
6428/6428 [==============================] - 6s - loss: 0.0117 - val_loss: 0.0099
Epoch 2/3
6428/6428 [==============================] - 5s - loss: 0.0099 - val_loss: 0.0098
Epoch 3/3
6428/6428 [==============================] - 5s - loss: 0.0095 - val_loss: 0.0096
